In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from nmnist_dataset import NMNISTDataset, augment # Import NMNIST dataset.

In [2]:
trained_folder = "Trained"
os.makedirs(trained_folder, exist_ok=True)

# Create SLAYER network

In [3]:
import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer.loss import SpikeMax
from lava.lib.dl.slayer.utils import Assistant

In [4]:
class SLAYERNetwork(torch.nn.Module):
    def __init__(self):
        super(SLAYERNetwork, self).__init__()
        
        neuron_params = {
            "threshold": 1.25,
            "current_decay": 0.25,
            "voltage_decay": 0.03,
            "tau_grad": 0.03,
            "scale_grad": 3,
            "requires_grad": True
        }
        neuron_params = {**neuron_params}
        
        self.blocks = torch.nn.ModuleList([
            slayer.block.cuba.Dense(neuron_params, 34*34*2, 512, weight_norm=True, delay=True),
            slayer.block.cuba.Dense(neuron_params, 512, 512, weight_norm=True, delay=True),
            slayer.block.cuba.Dense(neuron_params, 512, 10, weight_norm=True)
        ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike
    
    def grad_flow(self, path):
        # helps monitor the gradient flow.
        grad = [b.synapse.grad_norm for b in sef.blocks if hasattr(b, "synapse")]
        plt.figure()
        plt.semilogy(grad)
        plt.savefig(path+"gradFlow.png")
        plt.close()
        
        return grad

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

slayer_net = SLAYERNetwork().to(device)
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction="sum").to(device)
optimizer = torch.optim.Adam(slayer_net.parameters(), lr=0.001)
slayer_stats = slayer.utils.LearningStats()
slayer_assistant = Assistant(slayer_net, error, optimizer, slayer_stats, 
                             classifier=slayer.classifier.Rate.predict, count_log=False)

# Instantiate Dataset and DataLoader

In [6]:
training_set = NMNISTDataset(train=True, transform=augment, download=True)
testing_set = NMNISTDataset(train=False)
train_loader = DataLoader(dataset=training_set, batch_size=256, shuffle=True)
test_loader = DataLoader(dataset=testing_set, batch_size=256, shuffle=True)


NMNIST dataset is freely available here:
https://www.garrickorchard.com/datasets/n-mnist
(c) Creative Commons:
    Orchard, G.; Cohen, G.; Jayawant, A.; and Thakor, N.
    "Converting Static Image Datasets to Spiking Neuromorphic Datasets Using
    Saccades",
    Frontiers in Neuroscience, vol.9, no.437, Oct. 2015



# Train SLAYER Network

In [7]:
epochs = 20
for epoch in range(epochs):
    train_iter = iter(train_loader)
    print(f"\r[Epoch {epoch:2d}/{epochs}] {slayer_stats}", end="")
    
    for i, (inputs, labels) in tqdm(enumerate(train_iter)): # Training loop.
        output = slayer_assistant.train(inputs, labels)
    
    test_iter = iter(test_loader)
    for i, (inputs, label) in enumerate(test_iter): # Testing loop.
        output = slayer_assistant.test(inputs, label)
    
    print(f"\r[Epoch {epoch:2d}/{epochs}] {slayer_stats}", end="")
    
    if epoch%5 == 4: # Cleanup Display.
        print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {slayer_stats}'))
        stats_str = str(slayer_stats).replace("| ", "\n")
        print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')
    
    if slayer_stats.testing.best_accuracy:
        torch.save(slayer_net.state_dict(), trained_folder + '/slayer_network.pt')
    slayer_stats.update()

[Epoch  0/20] Train 

235it [02:56,  1.33it/s]


[Epoch  1/20] Train 

235it [02:55,  1.34it/s]


[Epoch  2/20] Train 

...

[Epoch 18/20] Train 

235it [02:55,  1.34it/s]


[Epoch 19/20] Train 

235it [02:55,  1.34it/s]


                                                                                                                                                                                       
[Epoch 19/20]
Train loss =     1.02871 (min =     1.01840)    accuracy = 0.95945 (max = 0.95863) 
Test  loss =     0.90141 (min =     0.84850)    accuracy = 0.97260 (max = 0.97100)
